<a href="https://colab.research.google.com/github/blood097/Work_examples/blob/main/Cell_lenght_PyImageJ_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Создание окружения для работы ImageJ в Gogle Colab**
---
⛔ **Опасная зона!**
<br>Пожалуйста, не изменяйте и не переносити фрагменты кода из этого раздела.

In [ ]:
# install JDK8 in colab
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
!apt update install_java()

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `apt update install_java()'


In [ ]:
!sudo apt-get install maven

Reading package lists... Done
Building dependency tree       
Reading state information... Done
maven is already the newest version (3.6.3-1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [ ]:
!pip install Cython pyimagej

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://anaconda.org/conda-forge/pyjnius/1.2.0/download/osx-64/pyjnius-1.2.0-py37h90b5fae_0.tar.bz2 && tar -xjf pyjnius-1.2.0-py37h90b5fae_0.tar.bz2 -C /usr/local
# set env
os.environ["PYJNIUS_JAR"] = "/usr/local/share/pyjnius/pyjnius.jar"

--2023-02-27 13:06:48--  https://anaconda.org/conda-forge/pyjnius/1.2.0/download/osx-64/pyjnius-1.2.0-py37h90b5fae_0.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.92.24, 104.17.93.24, 2606:4700::6811:5d18, ...
Connecting to anaconda.org (anaconda.org)|104.17.92.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5b5200910219ba13273541e3/5c6004c0d9c2cf34c4d1625c?response-content-disposition=attachment%3B%20filename%3D%22pyjnius-1.2.0-py37h90b5fae_0.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27pyjnius-1.2.0-py37h90b5fae_0.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=600&X-Amz-Date=20230227T130648Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjELP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIQCu8rfG4enHM5L21vdhkrvNTt2yUUU0K9UrIJI%2BujZ%2FvgIgVqAiOV3DTrZs5nxD%2FXaZPHuRLXGYs1Y0JkSzbLNTd1cqswUIWxAAGgw0NTU4NjQwOTgzNzgiDBWY7W23DFZctc

In [ ]:
import imagej
ij = imagej.init('sc.fiji:fiji')
print(ij.getVersion())

2.9.0/1.53t


# **Автоматизированный анализ изображений средствами PyImageJ**

In [ ]:
from scyjava import jimport

# run ImageJ commands
imp = ij.IJ.openImage("/content/SGEFix--8_1026_3w_x10_2.lsm");
ij.py.run_plugin(plugin="Set Measurements...", args="area feret's stack redirect=None decimal=3");
ij.py.run_plugin(imp=imp, plugin="8-bit", args="");
ij.IJ.setAutoThreshold(imp, "Default dark");
ij.py.run_plugin(imp=imp, plugin="Convert to Mask", args="method=Triangle background=Dark calculate");
ij.py.run_plugin(imp=imp, plugin="Analyze Particles...", args="size=0.3-2.5 circularity=0.00-0.45 show=Outlines display clear overlay stack");
ij.IJ.saveAs('Results', "/content/Results.csv");

[java.lang.Enum.toString]  	Area	Feret	Ch	FeretX	FeretY	FeretAngle	MinFeret[java.lang.Enum.toString] 
[java.lang.Enum.toString] 1	1.920	3.284	1	1038	107	20.556	1.922[java.lang.Enum.toString] 
[java.lang.Enum.toString] 2	1.477	2.461	1	1742	106	128.660	1.734[java.lang.Enum.toString] 
[java.lang.Enum.toString] 3	0.886	1.922	1	1043	112	36.870	1.279[java.lang.Enum.toString] 
[java.lang.Enum.toString] 4	1.920	3.261	1	943	115	45.000	1.648[java.lang.Enum.toString] 
[java.lang.Enum.toString] 5	1.329	2.798	1	1020	109	164.055	0.769[java.lang.Enum.toString] 
[java.lang.Enum.toString] 6	0.591	1.719	1	939	122	63.435	0.769[java.lang.Enum.toString] 
[java.lang.Enum.toString] 7	1.625	2.338	1	1030	129	170.538	1.702[java.lang.Enum.toString] 
[java.lang.Enum.toString] 8	1.773	2.578	1	1308	130	153.435	1.537[java.lang.Enum.toString] 
[java.lang.Enum.toString] 9	0.886	2.718	1	1320	134	135.000	0.815[java.lang.Enum.toString] 
[java.lang.Enum.toString] 10	0.591	2.174	1	1010	140	45.000	0.544[java.lang.Enum.toStr

# **Статистическая обработка результатов морфометрического анализа**